In [ ]:
import torch
from data_module import DataModule
from utility.utility import get_args
import numpy as np
from models.nodf import NODF
import nibabel as nib
from models.posterior import FVRF
from utility.utility import get_mask

%load_ext autoreload
%autoreload 2

## Args

In [ ]:
args = get_args(cmd=False)
# TODO: modify arguments here if needed
# args.ckpt_path = "..."

## Data

In [ ]:
data_module = DataModule(args)
data_module.setup("fit")

In [ ]:
dataset = data_module.dataset
dataloader = data_module.train_dataloader()
coords = dataloader.dataset.coords
coords.shape

In [ ]:
batch = next(iter(dataloader))
batch

## Model

In [ ]:
if args.ckpt_path:
    print("Loading model from checkpoint")
    model = NODF.load_from_checkpoint(args.ckpt_path).cpu()
else:
    model = NODF(args)

In [ ]:
model

In [ ]:
model.count_parameters()

## Forward Pass

In [ ]:
# ODF coefficients
chat = model(batch)
chat.shape

## Posterior

In [ ]:
# posterior = FVRF(args)

In [ ]:
# get roi
mask = get_mask(args)
# axial
mask[:168] = False
mask[169:] = False
# sagittal
mask[:, :74] = False
mask[:, 88:] = False
# coronal
mask[:, :, :67] = False
mask[:, :, 85:] = False

In [ ]:
# generate posterior samples
# post_samples_chat = posterior.sample_posterior(mask)

## Bayesian Optimizatoin

In [ ]:
from ax.service.ax_client import AxClient
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate

init_notebook_plotting()

In [ ]:
# TODO: set path
bo_client_path = "XXX"
map_ax_client = AxClient.load_from_json_file(filepath=bo_client_path)

In [ ]:
# show best trial
map_ax_client.get_best_trial()

In [ ]:
# fit model that predicts the hypter parameters
map_ax_client.fit_model()

In [ ]:
# plot hyperparameter distributions
render(map_ax_client.get_contour_plot(param_x='n_levels', param_y='base_resolution'))

In [ ]:
# plot objective line plot
best_objectives = np.array([[trial.objective_mean for trial in map_ax_client.experiment.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=best_objectives,
    title="Model performance vs. # of iterations",
    ylabel="Accuracy",
)
render(best_objective_plot)